In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import Dataset

import torchvision
import torchvision.transforms as transforms

import os
import time
import datetime
from models import ResNet18
from advertorch.attacks import L2PGDAttack

In [2]:
learning_rate = 0.1
file_name = 'basic_training_with_non_robust_dataset'

device = 'cuda' if torch.cuda.is_available() else 'cpu'

transform_train = transforms.Compose([transforms.ToTensor(),])
transform_test = transforms.Compose([transforms.ToTensor(),])

In [3]:
class TensorDataset(Dataset):
    def __init__(self, *tensors, transform=None):
        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)
        self.tensors = tensors
        self.transform = transform

    def __getitem__(self, index):
        im, targ = tuple(tensor[index] for tensor in self.tensors)
        if self.transform:
            real_transform = transforms.Compose([
                transforms.ToPILImage(),
                self.transform
            ])
            im = real_transform(im)
        return im, targ

    def __len__(self):
        return self.tensors[0].size(0)


In [4]:
data_path = "d_non_robust_CIFAR/"
train_data = torch.cat(torch.load(os.path.join(data_path, f"CIFAR_ims")))
train_labels = torch.cat(torch.load(os.path.join(data_path, f"CIFAR_lab")))
train_dataset = TensorDataset(train_data, train_labels, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=0)

Files already downloaded and verified


In [5]:
net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

adversary = L2PGDAttack(net, loss_fn=nn.CrossEntropyLoss(), eps=0.25, nb_iter=100, eps_iter=0.01, rand_init=True, clip_min=0.0, clip_max=1.0, targeted=False)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)

In [6]:
def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 10 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)

def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    benign_loss = 0
    adv_loss = 0
    benign_correct = 0
    adv_correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss = criterion(outputs, targets)
        benign_loss += loss.item()

        _, predicted = outputs.max(1)
        benign_correct += predicted.eq(targets).sum().item()

        if batch_idx % 10 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign test accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign test loss:', loss.item())

        adv = adversary.perturb(inputs, targets)
        adv_outputs = net(adv)
        loss = criterion(adv_outputs, targets)
        adv_loss += loss.item()

        _, predicted = adv_outputs.max(1)
        adv_correct += predicted.eq(targets).sum().item()

        if batch_idx % 10 == 0:
            print('Current adversarial test accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current adversarial test loss:', loss.item())

    print('\nTotal benign test accuarcy:', 100. * benign_correct / total)
    print('Total adversarial test Accuarcy:', 100. * adv_correct / total)
    print('Total benign test loss:', benign_loss)
    print('Total adversarial test loss:', adv_loss)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')

def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 100:
        lr /= 10
    if epoch >= 150:
        lr /= 10
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [7]:
t1 = time.time()
for epoch in range(0, 50):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)
t2 = time.time()


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.0703125
Current benign train loss: 2.555821657180786

Current batch: 10
Current benign train accuracy: 0.09375
Current benign train loss: 4.0017619132995605

Current batch: 20
Current benign train accuracy: 0.140625
Current benign train loss: 2.6469998359680176

Current batch: 30
Current benign train accuracy: 0.125
Current benign train loss: 2.452108144760132

Current batch: 40
Current benign train accuracy: 0.1953125
Current benign train loss: 2.09793758392334

Current batch: 50
Current benign train accuracy: 0.125
Current benign train loss: 2.0841498374938965

Current batch: 60
Current benign train accuracy: 0.1875
Current benign train loss: 2.1422271728515625

Current batch: 70
Current benign train accuracy: 0.1796875
Current benign train loss: 2.1242141723632812

Current batch: 80
Current benign train accuracy: 0.2421875
Current benign train loss: 1.9971405267715454

Current batch: 90
Current benign train accur


Current batch: 180
Current benign train accuracy: 0.5
Current benign train loss: 1.4900580644607544

Current batch: 190
Current benign train accuracy: 0.5078125
Current benign train loss: 1.281524419784546

Current batch: 200
Current benign train accuracy: 0.4296875
Current benign train loss: 1.4587613344192505

Current batch: 210
Current benign train accuracy: 0.6015625
Current benign train loss: 1.2042824029922485

Current batch: 220
Current benign train accuracy: 0.515625
Current benign train loss: 1.4009050130844116

Current batch: 230
Current benign train accuracy: 0.5625
Current benign train loss: 1.0914084911346436

Current batch: 240
Current benign train accuracy: 0.5390625
Current benign train loss: 1.2506885528564453

Current batch: 250
Current benign train accuracy: 0.5390625
Current benign train loss: 1.1781357526779175

Current batch: 260
Current benign train accuracy: 0.515625
Current benign train loss: 1.3372753858566284

Current batch: 270
Current benign train accuracy


Current batch: 350
Current benign train accuracy: 0.6875
Current benign train loss: 0.9707896709442139

Current batch: 360
Current benign train accuracy: 0.609375
Current benign train loss: 1.0231746435165405

Current batch: 370
Current benign train accuracy: 0.71875
Current benign train loss: 0.8950746655464172

Current batch: 380
Current benign train accuracy: 0.734375
Current benign train loss: 0.8229987025260925

Current batch: 390
Current benign train accuracy: 0.7125
Current benign train loss: 0.8415331840515137

Total benign train accuarcy: 62.598
Total benign train loss: 405.62482446432114

[ Test epoch: 2 ]

Current batch: 0
Current benign test accuracy: 0.5
Current benign test loss: 1.4906924962997437
Current adversarial test accuracy: 0.29
Current adversarial test loss: 2.3687243461608887

Current batch: 10
Current benign test accuracy: 0.53
Current benign test loss: 1.4183473587036133
Current adversarial test accuracy: 0.32
Current adversarial test loss: 2.2029733657836914

Current adversarial test accuracy: 0.32
Current adversarial test loss: 2.1748335361480713

Current batch: 70
Current benign test accuracy: 0.5
Current benign test loss: 1.429205298423767
Current adversarial test accuracy: 0.32
Current adversarial test loss: 2.3821005821228027

Current batch: 80
Current benign test accuracy: 0.6
Current benign test loss: 1.1779156923294067
Current adversarial test accuracy: 0.34
Current adversarial test loss: 1.9852783679962158

Current batch: 90
Current benign test accuracy: 0.56
Current benign test loss: 1.1753947734832764
Current adversarial test accuracy: 0.32
Current adversarial test loss: 1.9902938604354858

Total benign test accuarcy: 53.81
Total adversarial test Accuarcy: 30.23
Total benign test loss: 132.1948093175888
Total adversarial test loss: 219.6860740184784
Model Saved!

[ Train epoch: 4 ]

Current batch: 0
Current benign train accuracy: 0.7578125
Current benign train loss: 0.6354665756225586

Current batch: 10
Current benign train accur


Current batch: 100
Current benign train accuracy: 0.8203125
Current benign train loss: 0.5247474908828735

Current batch: 110
Current benign train accuracy: 0.8203125
Current benign train loss: 0.5660082697868347

Current batch: 120
Current benign train accuracy: 0.8046875
Current benign train loss: 0.5377472639083862

Current batch: 130
Current benign train accuracy: 0.7890625
Current benign train loss: 0.5485981106758118

Current batch: 140
Current benign train accuracy: 0.875
Current benign train loss: 0.4059702754020691

Current batch: 150
Current benign train accuracy: 0.796875
Current benign train loss: 0.5294559597969055

Current batch: 160
Current benign train accuracy: 0.796875
Current benign train loss: 0.6583704352378845

Current batch: 170
Current benign train accuracy: 0.8515625
Current benign train loss: 0.47033149003982544

Current batch: 180
Current benign train accuracy: 0.8046875
Current benign train loss: 0.5261343121528625

Current batch: 190
Current benign train a

...

Current adversarial test accuracy: 0.31
Current adversarial test loss: 5.1627607345581055

Current batch: 10
Current benign test accuracy: 0.63
Current benign test loss: 1.6044065952301025
Current adversarial test accuracy: 0.27
Current adversarial test loss: 5.045785903930664

Current batch: 20
Current benign test accuracy: 0.65
Current benign test loss: 1.427030086517334
Current adversarial test accuracy: 0.34
Current adversarial test loss: 4.766022205352783

Current batch: 30
Current benign test accuracy: 0.7
Current benign test loss: 1.3020092248916626
Current adversarial test accuracy: 0.35
Current adversarial test loss: 4.355163097381592

Current batch: 40
Current benign test accuracy: 0.74
Current benign test loss: 1.1852734088897705
Current adversarial test accuracy: 0.31
Current adversarial test loss: 4.570682048797607

Current batch: 50
Current benign test accuracy: 0.68
Current benign test loss: 1.3111308813095093
Current adversarial test accuracy: 0.35
Current adversarial t


Total benign test accuarcy: 70.03
Total adversarial test Accuarcy: 31.14
Total benign test loss: 123.24982100725174
Total adversarial test loss: 436.113486289978
Model Saved!

[ Train epoch: 45 ]

Current batch: 0
Current benign train accuracy: 0.9765625
Current benign train loss: 0.06395556032657623

Current batch: 10
Current benign train accuracy: 0.9921875
Current benign train loss: 0.034517090767621994

Current batch: 20
Current benign train accuracy: 0.96875
Current benign train loss: 0.07114967703819275

Current batch: 30
Current benign train accuracy: 1.0
Current benign train loss: 0.009012758731842041

Current batch: 40
Current benign train accuracy: 0.984375
Current benign train loss: 0.04412161931395531

Current batch: 50
Current benign train accuracy: 0.984375
Current benign train loss: 0.07644479721784592

Current batch: 60
Current benign train accuracy: 0.96875
Current benign train loss: 0.05970684438943863

Current batch: 70
Current benign train accuracy: 0.9765625
Curre


Current batch: 150
Current benign train accuracy: 0.984375
Current benign train loss: 0.06540238857269287

Current batch: 160
Current benign train accuracy: 0.96875
Current benign train loss: 0.06287256628274918

Current batch: 170
Current benign train accuracy: 0.9609375
Current benign train loss: 0.09583903104066849

Current batch: 180
Current benign train accuracy: 0.9921875
Current benign train loss: 0.041169144213199615

Current batch: 190
Current benign train accuracy: 0.984375
Current benign train loss: 0.08787960559129715

Current batch: 200
Current benign train accuracy: 0.9765625
Current benign train loss: 0.09151912480592728

Current batch: 210
Current benign train accuracy: 1.0
Current benign train loss: 0.01409129612147808

Current batch: 220
Current benign train accuracy: 0.96875
Current benign train loss: 0.058376822620630264

Current batch: 230
Current benign train accuracy: 0.984375
Current benign train loss: 0.03404507786035538

Current batch: 240
Current benign trai


Current batch: 320
Current benign train accuracy: 0.9765625
Current benign train loss: 0.05966367572546005

Current batch: 330
Current benign train accuracy: 0.984375
Current benign train loss: 0.04737755283713341

Current batch: 340
Current benign train accuracy: 0.9765625
Current benign train loss: 0.06941618025302887

Current batch: 350
Current benign train accuracy: 0.984375
Current benign train loss: 0.03862413018941879

Current batch: 360
Current benign train accuracy: 0.984375
Current benign train loss: 0.041451651602983475

Current batch: 370
Current benign train accuracy: 0.9765625
Current benign train loss: 0.09769408404827118

Current batch: 380
Current benign train accuracy: 0.9296875
Current benign train loss: 0.18392260372638702

Current batch: 390
Current benign train accuracy: 0.95
Current benign train loss: 0.1955200731754303

Total benign train accuarcy: 97.554
Total benign train loss: 27.52940155286342

[ Test epoch: 47 ]

Current batch: 0
Current benign test accura

Current adversarial test accuracy: 0.29
Current adversarial test loss: 5.118072032928467

Current batch: 50
Current benign test accuracy: 0.65
Current benign test loss: 1.4201252460479736
Current adversarial test accuracy: 0.27
Current adversarial test loss: 5.245668888092041

Current batch: 60
Current benign test accuracy: 0.62
Current benign test loss: 1.3553369045257568
Current adversarial test accuracy: 0.23
Current adversarial test loss: 5.332019805908203

Current batch: 70
Current benign test accuracy: 0.7
Current benign test loss: 1.7171627283096313
Current adversarial test accuracy: 0.3
Current adversarial test loss: 5.168315887451172

Current batch: 80
Current benign test accuracy: 0.66
Current benign test loss: 1.6325033903121948
Current adversarial test accuracy: 0.28
Current adversarial test loss: 5.207157611846924

Current batch: 90
Current benign test accuracy: 0.67
Current benign test loss: 1.6032315492630005
Current adversarial test accuracy: 0.27
Current adversarial te

In [8]:
#RTX3070 50 epochs
deltatime = str(datetime.timedelta(seconds=t2-t1))
print('Total running time is %s '% deltatime)

Total running time is 5:03:25.954485 
